In [59]:
! pip install pickle
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import pickle


ERROR: Could not find a version that satisfies the requirement pickle (from versions: none)
ERROR: No matching distribution found for pickle


In [60]:
## load the data set
data=pd.read_csv("Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [61]:
#pre Process the data
## Drop Irrelevent Columns
data=data.drop(['RowNumber','CustomerId','Surname'],axis=1)
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [62]:
## Encode Categorical Variables for binary classification like 0 or 1 like gender F is concedure 0 and M is 1
label_encoder_gender=LabelEncoder()
data['Gender']=label_encoder_gender.fit_transform(data['Gender'])
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [63]:
#one hot encoding Geograpghy  for multi classification values contain in column then use one hot encoding

one_hot_encode=OneHotEncoder()
geo_encode=one_hot_encode.fit_transform(data[['Geography']])
geo_encode


<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [64]:
one_hot_encode.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [65]:
geo_encoded_df=pd.DataFrame(geo_encode.toarray(),columns=one_hot_encode.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [66]:
#combine all the colums
data=pd.concat([data.drop('Geography',axis=1),geo_encoded_df],axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [67]:
#save the encoders and SScaler
with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)

with open('one_hot_encode.pkl','wb') as file:
    pickle.dump(one_hot_encode,file)

    

In [78]:
## divide the dataset into independant and dependant fetures
X=data.drop('Exited',axis=1)
Y=data['Exited']

## Split the data in training and testing sets
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=42)

### Scale this fetures
Scalar=StandardScaler()
X_train=Scalar.fit_transform(X_train)
X_test =Scalar.transform(X_test)


In [79]:
X_train

array([[ 0.35649971,  0.91324755, -0.6557859 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [-0.20389777,  0.91324755,  0.29493847, ..., -0.99850112,
         1.72572313, -0.57638802],
       [-0.96147213,  0.91324755, -1.41636539, ..., -0.99850112,
        -0.57946723,  1.73494238],
       ...,
       [ 0.86500853, -1.09499335, -0.08535128, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.15932282,  0.91324755,  0.3900109 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.47065475,  0.91324755,  1.15059039, ..., -0.99850112,
         1.72572313, -0.57638802]])

In [80]:
with open('Scalar.pkl','wb') as file:
    pickle.dump(Scalar,file)

ANN IMPLIMENTATION

In [71]:

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime
(X_train.shape[1],)

(12,)

In [72]:
model=Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)),###HL1(hedan layer first ) connected with input layer
    Dense(32,activation='relu'),##HL2
    Dense(1,activation='sigmoid')## output layer(sigmoid activation function becouse binary classification if its multi calla that will be use softmax)
    
    ])

In [73]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 64)                832       
                                                                 
 dense_9 (Dense)             (None, 32)                2080      
                                                                 
 dense_10 (Dense)            (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [74]:
import tensorflow
opt=tensorflow.keras.optimizers.Adam(learning_rate=0.01)
loss=tensorflow.keras.losses.BinaryCrossentropy()

In [75]:
### complie the model
model.compile(optimizer=opt,loss='binary_crossentropy',metrics=['accuracy'])

In [76]:
## setup tensor board
from tensorflow.keras.callbacks import EarlyStopping ,TensorBoard
log_dir="logs/fit/"+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [77]:
## set up early stoping
early_stoping_callback=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)


In [53]:
### Training the model
histroy=model.fit(X_train,Y_train,validation_data=(X_test,Y_test),epochs=100,
                  callbacks=[tensorflow_callback,early_stoping_callback])

Epoch 1/100
250/250 [==============================] - 1s 6ms/step - loss: 0.3130 - accuracy: 0.8690 - val_loss: 0.3416 - val_accuracy: 0.8660
Epoch 2/100
250/250 [==============================] - 1s 5ms/step - loss: 0.3133 - accuracy: 0.8708 - val_loss: 0.3558 - val_accuracy: 0.8630
Epoch 3/100
250/250 [==============================] - 1s 5ms/step - loss: 0.3096 - accuracy: 0.8715 - val_loss: 0.3609 - val_accuracy: 0.8615
Epoch 4/100
250/250 [==============================] - 1s 5ms/step - loss: 0.3067 - accuracy: 0.8716 - val_loss: 0.3572 - val_accuracy: 0.8555
Epoch 5/100
250/250 [==============================] - 1s 5ms/step - loss: 0.3063 - accuracy: 0.8684 - val_loss: 0.3548 - val_accuracy: 0.8595
Epoch 6/100
250/250 [==============================] - 1s 5ms/step - loss: 0.3024 - accuracy: 0.8712 - val_loss: 0.3614 - val_accuracy: 0.8605
Epoch 7/100
250/250 [==============================] - 1s 5ms/step - loss: 0.3014 - accuracy: 0.8744 - val_loss: 0.3667 - val_accuracy: 0.8615

In [54]:
model.save('model.h5')

f:\study material\NLP_Rag _Projects\ANN_Implimentation\.venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [55]:
## Load tensorboard Extention
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [56]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 7132), started 0:06:49 ago. (Use '!kill 7132' to kill it.)

In [ ]:
### load pkl file
